In [1]:
import cv2, numpy as np, glob, os, time, matplotlib.pyplot as plt
import pyrealsense2 as rs
from scipy.spatial.transform import Rotation as R
import rby1_sdk as rb
import json

In [2]:
SAVE_DIR = "calib_data"
IMG_DIR = os.path.join(SAVE_DIR,"images"); os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)
K_FILE = "K_1280x720.npy"
D_FILE = "D_1280x720.npy"
PRESET_FILE = "rs_locked_preset.json"
ADDRESS = "192.168.30.1:50051"
MODEL = "a"
POWER = ".*"
BASE_INDEX, LINK_TORSO_5_INDEX, EE_RIGHT_INDEX, EE_LEFT_INDEX = 0, 1, 2, 3

In [4]:
def get_joint_positions() -> list:
    robot = rb.create_robot(ADDRESS, MODEL)
    robot.connect()
    if not robot.is_connected():
        print("Robot is not connected")
        exit(1)
    if not robot.is_power_on(POWER):
        rv = robot.power_on(POWER)
        if not rv:
            print("Failed to power on")
            exit(1)
    robot.reset_fault_control_manager()
    robot.enable_control_manager()

    robot_state = robot.get_state()
    body = [round(x, 3) for x in robot_state.position[2:8]]
    right_arm = [round(x, 3) for x in robot_state.position[8:15]]
    left_arm = [round(x, 3) for x in robot_state.position[15:22]]

    result = [
        body,
        right_arm,
        left_arm
    ]
    return result

def get_flange_homogeneous() -> np.array:
    robot = rb.create_robot(ADDRESS, MODEL)
    model = robot.model()
    robot.connect()
    
    dyn_model = robot.get_dynamics()
    dyn_state = dyn_model.make_state(["base", "link_torso_5", "ee_right", "ee_left"], model.robot_joint_names)
    dyn_state.set_q(robot.get_state().position)
    dyn_model.compute_forward_kinematics(dyn_state)
    T_base2left = dyn_model.compute_transformation(dyn_state, BASE_INDEX, EE_LEFT_INDEX)
    
    print(T_base2left)

    return T_base2left

def matrix_to_quat_xyzw(Rm):
    return R.from_matrix(Rm).as_quat().tolist()

In [7]:
def load_KD(k_path, d_path):
    try:
        K = np.load(k_path)
    except Exception:
        base = os.path.splitext(k_path)[0]
        try:
            K = np.load(base + ".npy")
        except Exception:
            raise RuntimeError(f"Cannot load K from {k_path}")
    try:
        D = np.load(d_path)
    except Exception:
        base = os.path.splitext(d_path)[0]
        try:
            D = np.load(base + "_D.npy")
        except Exception:
            raise RuntimeError(f"Cannot load D from {d_path}")
    return K, D

try:
    K_mat, D_vec = load_KD(K_FILE, D_FILE)
except Exception as e:
    print("Warning: cannot load K/D now:", e)
    K_mat, D_vec = None, None

IMG_DIR = "calib_images"
SAVE_DIR = "calib_records"
cols, rows, spacing = 4, 5, 0.02
DISPLAY_SCALE = 0.50
UNDISTORT_FOR_DETECT = False
K_FILE = "K_1280x720.npy"
D_FILE = "K_1280x720.npy"
PRESET_FILE = "rs_locked_preset.json"
SAVE_DEBUG_IMAGES = True
PATCH_DIR = "point_patches"
DEBUG_DIR = "debug_images"


os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(PATCH_DIR, exist_ok=True)
os.makedirs(DEBUG_DIR, exist_ok=True)


K_mat, D_vec = load_KD(K_FILE, D_FILE)


In [8]:

params = cv2.SimpleBlobDetector_Params()
params.minThreshold=10; params.maxThreshold=220; params.thresholdStep=10
params.filterByArea = True; params.minArea = 30; params.maxArea = 50000
params.filterByCircularity = True; params.minCircularity = 0.6
params.filterByInertia = True; params.minInertiaRatio = 0.2
params.filterByConvexity = True; params.minConvexity = 0.6
params.filterByColor = True; params.blobColor = 255
detector = cv2.SimpleBlobDetector_create(params)

pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
profile = pipeline.start(cfg)
time.sleep(0.1)
align_to = rs.stream.color
align = rs.align(align_to)

dev = profile.get_device()
depth_sensor = dev.first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("[INFO] depth scale (m/unit):", depth_scale)

def get_depth_m_at_pixel(depth_image, u, v, depth_scale, init_patch=3, max_patch=21):
    h, w = depth_image.shape
    u0 = int(round(u)); v0 = int(round(v))
    for r in range(init_patch, max_patch+1, 2):
        x1 = max(0, u0 - r); x2 = min(w-1, u0 + r)
        y1 = max(0, v0 - r); y2 = min(h-1, v0 + r)
        patch = depth_image[y1:y2+1, x1:x2+1].astype(np.float32)
        valid = patch > 0
        if valid.sum() > 0:
            med = np.median(patch[valid]) * depth_scale
            return float(med), (x1,x2,y1,y2)
    return None, None

def pixel_to_cam_point(u, v, z, K):
    fx = float(K[0,0]); fy = float(K[1,1]); cx = float(K[0,2]); cy = float(K[1,2])
    x = (u - cx) * z / fx
    y = (v - cy) * z / fy
    return np.array([x, y, z], dtype=float)

def campoint_to_grip_base(p_cam, H_base_grip=None, cam2grip_R=None, cam2grip_t=None):
    out = {}
    if cam2grip_R is not None and cam2grip_t is not None:
        p_grip = cam2grip_R.dot(p_cam) + cam2grip_t
        out["p_grip"] = p_grip.tolist()
        if H_base_grip is not None:
            Rbg = H_base_grip[:3,:3]; tbg = H_base_grip[:3,3]
            p_base = Rbg.dot(p_grip) + tbg
            out["p_base"] = p_base.tolist()
    return out

def make_objp_for_pattern(pattern_type):
    if pattern_type == "asymmetric":
        objp = np.zeros((cols*rows,3), np.float64)
        idxp = 0
        for j in range(rows):
            for i in range(cols):
                x = (2 * i + (j % 2)) * spacing
                y = j * spacing
                objp[idxp,0] = x; objp[idxp,1] = y; idxp += 1
        return objp
    else:
        objp = np.zeros((cols*rows,3), np.float64)
        grid = np.mgrid[0:cols, 0:rows].T.reshape(-1,2)
        objp[:,:2] = grid * spacing
        return objp

idx = len([n for n in os.listdir(IMG_DIR) if n.endswith(".png")])
print("Press 's' to save frame+pose, 'q' to quit.")
try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)
        color_frame = aligned.get_color_frame()
        depth_frame = aligned.get_depth_frame()
        if not color_frame or not depth_frame:
            continue
        color = np.asanyarray(color_frame.get_data())
        depth = np.asanyarray(depth_frame.get_data())

        disp = color.copy()
        if DISPLAY_SCALE != 1.0:
            disp = cv2.resize(disp, (int(disp.shape[1]*DISPLAY_SCALE), int(disp.shape[0]*DISPLAY_SCALE)))
        cv2.imshow("cap", disp)
        k = cv2.waitKey(1) & 0xFF
        if k == ord('s'):
            t_wall = time.time()
            t_mon = time.monotonic_ns()
            img_name = f"he_img_{idx:04d}.png"
            img_path = os.path.join(IMG_DIR, img_name)
            cv2.imwrite(img_path, color)

            joints = get_joint_positions()
            H = get_flange_homogeneous()
            H_list = H.tolist()
            t_vec = H[:3,3].tolist()
            quat = matrix_to_quat_xyzw(H[:3,:3])

            gray = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)
            gray_for_detect = gray
            if UNDISTORT_FOR_DETECT and (K_mat is not None and D_vec is not None):
                newK, _ = cv2.getOptimalNewCameraMatrix(K_mat, D_vec, (gray.shape[1], gray.shape[0]), 0)
                gray_for_detect = cv2.undistort(color, K_mat, D_vec, None, newK)
                gray_for_detect = cv2.cvtColor(gray_for_detect, cv2.COLOR_BGR2GRAY)

            board_info = {"found": False, "pattern_type": None, "imgpts2d": [], "depths_m": [], "points_cam": [], "rvec": None, "tvec": None, "inliers": None}
            centers = None

            try:
                ret_asym, centers_asym = cv2.findCirclesGrid(gray_for_detect, (cols, rows),
                                                             flags=cv2.CALIB_CB_ASYMMETRIC_GRID,
                                                             blobDetector=detector)
            except Exception:
                ret_asym = False; centers_asym = None
            if ret_asym:
                centers = centers_asym.reshape(-1,2).astype(np.float32)
                pattern_type = "asymmetric"
            else:
                try:
                    ret_sym, centers_sym = cv2.findCirclesGrid(gray_for_detect, (cols, rows),
                                                               flags=cv2.CALIB_CB_SYMMETRIC_GRID,
                                                               blobDetector=detector)
                except Exception:
                    ret_sym = False; centers_sym = None
                if ret_sym:
                    centers = centers_sym.reshape(-1,2).astype(np.float32)
                    pattern_type = "symmetric"

            if centers is not None:
                board_info["found"] = True
                board_info["pattern_type"] = pattern_type
                board_info["imgpts2d"] = centers.tolist()

                depths_list = []
                campts_list = []
                for i,(cx,cy) in enumerate(centers):
                    z_m, bbox = get_depth_m_at_pixel(depth, cx, cy, depth_scale, init_patch=3, max_patch=21)
                    if z_m is None:
                        depths_list.append(None)
                        campts_list.append(None)
                    else:
                        depths_list.append(z_m)
                        if K_mat is not None:
                            p_cam = pixel_to_cam_point(float(cx), float(cy), z_m, K_mat)
                            campts_list.append(p_cam.tolist())
                        else:
                            campts_list.append([None,None,z_m])
                    if SAVE_DEBUG_IMAGES and bbox is not None:
                        x1,x2,y1,y2 = bbox
                        patch = gray[y1:y2+1, x1:x2+1]
                        cv2.imwrite(os.path.join(PATCH_DIR, f"patch_{idx:04d}_pt{i:02d}.png"), patch)
                board_info["depths_m"] = depths_list
                board_info["points_cam"] = campts_list

                if K_mat is not None:
                    objp = make_objp_for_pattern(pattern_type)

                    try:
                        ok, rvec_r, tvec_r, inliers = cv2.solvePnPRansac(objp.astype(np.float64), centers.astype(np.float64), K_mat, D_vec,
                                                                         iterationsCount=200, reprojectionError=8.0, confidence=0.99)
                    except Exception:
                        ok = False; rvec_r = None; tvec_r = None; inliers = None
                    if ok:
                        board_info["rvec"] = rvec_r.reshape(3).tolist()
                        board_info["tvec"] = tvec_r.reshape(3).tolist()
                        board_info["inliers"] = int(len(inliers)) if inliers is not None else None
                    else:
                        try:
                            ok2, rvec2, tvec2 = cv2.solvePnP(objp.astype(np.float64), centers.astype(np.float64), K_mat, D_vec, flags=cv2.SOLVEPNP_ITERATIVE)
                            if ok2:
                                board_info["rvec"] = rvec2.reshape(3).tolist()
                                board_info["tvec"] = tvec2.reshape(3).tolist()
                        except Exception:
                            pass
            rec = {
                "image": img_name,
                "timestamp": t_wall,
                "monotonic_ns": t_mon,
                "camera_intrinsics": K_FILE,
                "preset": PRESET_FILE,
                "gripper_pose_base": {"H": H_list, "translation_m": t_vec, "rotation_quat_xyzw": quat},
                "joints": joints,
                "board_detection": board_info,
                "notes": ""
            }
            json_name = f"he_record_{idx:04d}.json"
            with open(os.path.join(SAVE_DIR, json_name), 'w') as f:
                json.dump(rec, f, indent=2)

            print("Saved", img_name, json_name, "found:", board_info["found"], "pattern:", board_info.get("pattern_type"))
            idx += 1

        elif k == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


[INFO] depth scale (m/unit): 0.0010000000474974513
Press 's' to save frame+pose, 'q' to quit.
[[ 0.16783197  0.13660083 -0.97630561  0.363763  ]
 [-0.72525042  0.68789808 -0.0284263   0.08974238]
 [ 0.6677157   0.7128369   0.2145211   1.24475732]
 [ 0.          0.          0.          1.        ]]
Saved he_img_0002.png he_record_0002.json found: True pattern: asymmetric
